In [ ]:

import optuna
from sklearn.model_selection import StratifiedGroupKFold
import pandas as pd
import joblib

# Global Configuration
validate_or_submit = 'submit'
verbose = True
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange, tqdm
import itertools
import warnings
import json
import os
from sklearn.base import ClassifierMixin, BaseEstimator, clone
from sklearn.model_selection import cross_val_predict, GroupKFold, train_test_split
from xgboost import XGBClassifier

from sklearn.metrics import f1_score
import gc
from scipy.ndimage import binary_closing, binary_opening, gaussian_filter1d, median_filter
from scipy.signal import savgol_filter



SEED=1234
os.environ["PYTHONHASHSEED"] = str(SEED)

rnd = np.random.RandomState(SEED)
np.random.seed(SEED)


def get_gpu_params():
    """Detect GPU and return XGBoost parameters."""
    try:
        import subprocess
        result = subprocess.run(['nvidia-smi'], capture_output=True)
        if result.returncode == 0:
            print('✓ GPU detected - using GPU acceleration')
            return {'tree_method': 'hist', 'device': 'cuda', 'predictor': 'gpu_predictor'}
    except:
        pass
    print('⚠ No GPU - using CPU')
    return {'tree_method': 'hist', 'predictor': 'cpu_predictor'}

GPU_PARAMS = get_gpu_params()
class CFG:
    BASE_PATH = "/kaggle/input/MABe-mouse-behavior-detection"
    train_path = f"{BASE_PATH}/train.csv"
    test_path = f"{BASE_PATH}/test.csv"
    train_annotation_path = f"{BASE_PATH}/train_annotation"
    train_tracking_path = f"{BASE_PATH}/train_tracking"
    test_tracking_path = f"{BASE_PATH}/test_tracking"

    model_path = "/kaggle/input/social-action-recognition-in-mice-xgb-catboost"
    model_name = "xgboost"
    MODEL_SAVE_PATH = "/kaggle/working/models"
    
    # mode = "validate"
    mode = "submit"

    model = XGBClassifier(
        verbosity=0, 
        random_state=42,
        n_estimators=700, 
        learning_rate=0.05, 
        max_depth=6,
        min_child_weight=5, 
        subsample=0.8, 
        colsample_bytree=0.8,
        **GPU_PARAMS  # GPU acceleration
    )

from catboost import CatBoostClassifier

# Cấu hình cho CatBoost
CAT_PARAMS = {
    'iterations': 700,         # Tương đương n_estimators
    'learning_rate': 0.05,
    'depth': 7,
    'loss_function': 'Logloss',
    'verbose': 0,
    'random_seed': 42,
    'task_type': 'GPU',        # Dùng GPU để train nhanh
    'devices': '0',
    'allow_writing_files': False
}

class DataSampler(ClassifierMixin, BaseEstimator):
    def __init__(self, estimator, neg_pos_ratio=10.0): 
        self.estimator = estimator
        self.neg_pos_ratio = neg_pos_ratio

    def fit(self, X, y):
        X_arr = np.asarray(X)
        y_arr = np.assarray(y)
        
        pos_indices = np.where(y_arr == 1)[0]
        neg_indices = np.where(y_arr == 0)[0]
        
        if len(pos_indices) == 0:
            self.estimator.fit(X_arr[::10], y_arr[::10])
            self.classes_ = self.estimator.classes_
            return self

        n_neg_keep = int(len(pos_indices) * self.neg_pos_ratio)
        
        if len(neg_indices) > n_neg_keep:
            kept_neg_indices = np.random.choice(neg_indices, n_neg_keep, replace=False)
        else:
            kept_neg_indices = neg_indices
            
        final_indices = np.concatenate([pos_indices, kept_neg_indices])
        np.random.shuffle(final_indices)
        
        self.estimator.fit(X_arr[final_indices], y_arr[final_indices])
        self.classes_ = self.estimator.classes_
        return self

    def predict_proba(self, X):
        return self.estimator.predict_proba(np.array(X))

    def predict(self, X):
        return self.estimator.predict(np.array(X))

def train_and_save_models(body_parts_tracked_str, switch_tr, X_tr, label, meta, section_id):
    """Train XGBoost and CatBoost models and save them to disk."""
    import xgboost
    import catboost
    
    # Create save directory
    save_dir = os.path.join(CFG.MODEL_SAVE_PATH, switch_tr, f"section_{section_id}")
    os.makedirs(save_dir, exist_ok=True)
    
    model_list = []
    
    for action in label.columns:
        action_mask = ~label[action].isna().values
        y_action = label[action][action_mask].values.astype(int)
        
        if not (y_action == 0).all() and len(np.unique(y_action)) >= 2:
            print(f"  Training models for action: {action}")
            
            # Train XGBoost
            model_xgb = DataSampler(clone(CFG.model), neg_pos_ratio=10.0)
            model_xgb.fit(X_tr[action_mask], y_action)
            
            # Train CatBoost
            cat_model = CatBoostClassifier(**CAT_PARAMS)
            model_cat = DataSampler(cat_model, neg_pos_ratio=10.0)
            model_cat.fit(X_tr[action_mask], y_action)
            
            # Save XGBoost model
            xgb_path = os.path.join(save_dir, f"{action}_xgboost.joblib")
            joblib.dump(model_xgb, xgb_path)
            print(f"    Saved XGBoost: {xgb_path}")
            
            cat_path = os.path.join(save_dir, f"{action}_catboost.cbm")
            model_cat.estimator.save_model(cat_path)
            joblib.dump(model_cat, os.path.join(save_dir, f"{action}_catboost_wrapper.joblib"))
            print(f"    Saved CatBoost: {cat_path}")
            
            model_list.append(action)
    
    metadata = {
        "feature_names": list(X_tr.columns),
        "actions": model_list,
        "body_parts_tracked_str": body_parts_tracked_str,
        "switch_type": switch_tr,
        "xgboost_version": xgboost.__version__,
        "catboost_version": catboost.__version__
    }
    
    metadata_path = os.path.join(save_dir, "metadata.json")
    with open(metadata_path, "w") as f:
        json.dump(metadata, f, indent=2)
    print(f"  Saved metadata: {metadata_path}")
    
    return model_list


In [ ]:
def safe_sqrt(x):
    """Robust sqrt that clips negative values to 0."""
    if isinstance(x, (pd.Series, pd.DataFrame)):
        return np.sqrt(x.clip(lower=0))
    return np.sqrt(np.maximum(x, 0))

# Suppress runtime warnings for invalid values in comparisons
warnings.filterwarnings('ignore', 'invalid value encountered')

def smooth_coordinates(df, sigma=1.5):
    """Apply Gaussian smoothing to coordinate columns."""
    df_smooth = df.copy()
    for col in df.columns:
        # Fill NaN before smoothing
        filled = df[col].interpolate(method='linear', limit_direction='both').fillna(0).values
        # Apply Gaussian filter
        smoothed = gaussian_filter1d(filled, sigma=sigma)
        df_smooth[col] = smoothed
    return df_smooth


def safe_rolling(series, window, func, min_periods=None):
    """Safe rolling operation with NaN handling"""
    if min_periods is None:
        min_periods = max(1, window // 4)
    return series.rolling(window, min_periods=min_periods, center=True).apply(func, raw=True)

def _scale(n_frames_at_30fps, fps, ref=30.0):
    """Scale a frame count defined at 30 fps to the current video's fps."""
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    """Signed version of _scale for forward/backward shifts (keeps at least 1 frame when |n|>=1)."""
    if n_frames_at_30fps == 0:
        return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

def _fps_from_meta(meta_df, fallback_lookup, default_fps=30.0):
    if 'frames_per_second' in meta_df.columns and pd.notnull(meta_df['frames_per_second']).any():
        return float(meta_df['frames_per_second'].iloc[0])
    vid = meta_df['video_id'].iloc[0]
    return float(fallback_lookup.get(vid, default_fps))



In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
import itertools

def safe_sqrt(x):
    """Robust sqrt that clips negative values to 0."""
    if isinstance(x, (pd.Series, pd.DataFrame)):
        return np.sqrt(x.clip(lower=0))
    return np.sqrt(np.maximum(x, 0))

def _scale(n_frames_at_30fps, fps, ref=30.0):
    """Scale a frame count defined at 30 fps to the current video's fps."""
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    """Signed version of _scale for forward/backward shifts (keeps at least 1 frame when |n|>=1)."""
    if n_frames_at_30fps == 0:
        return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

def add_curvature_features(X, center_x, center_y, fps):
    """Trajectory curvature (window lengths scaled by fps)."""
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()

    cross_prod = vel_x * acc_y - vel_y * acc_x
    vel_mag = safe_sqrt(vel_x**2 + vel_y**2)
    curvature = np.abs(cross_prod) / (vel_mag**3 + 1e-6)  # invariant to time scaling

    new_features = {}
    for w in [30, 60]:
        ws = _scale(w, fps)
        new_features[f'curv_mean_{w}'] = curvature.rolling(ws, min_periods=max(1, ws // 6)).mean()

    angle = np.arctan2(vel_y, vel_x)
    angle_change = np.abs(angle.diff())
    w = 30
    ws = _scale(w, fps)
    new_features[f'turn_rate_{w}'] = angle_change.rolling(ws, min_periods=max(1, ws // 6)).sum()

    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    return X

def add_multiscale_features(X, center_x, center_y, fps):
    """Multi-scale temporal features (speed in cm/s; windows scaled by fps)."""
    # displacement per frame is already in cm (pix normalized earlier); convert to cm/s
    speed = safe_sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    
    # Smooth speed signal using Savitzky-Golay filter (window ~15 frames)
    try:
        ws = min(15, len(speed) // 2)
        if ws >= 5 and ws % 2 == 1:  # Must be odd
            speed_smooth = pd.Series(savgol_filter(speed.fillna(0).values, ws, 3), index=speed.index)
            X['speed_smooth'] = speed_smooth
    except:
        pass
    

    new_features = {}
    scales = [10, 40, 160]
    for scale in scales:
        ws = _scale(scale, fps)
        if len(speed) >= ws:
            new_features[f'sp_m{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).mean()
            new_features[f'sp_s{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).var().clip(lower=0).pow(0.5)

    if len(scales) >= 2 and f'sp_m{scales[0]}' in new_features and f'sp_m{scales[-1]}' in new_features:
        new_features['sp_ratio'] = new_features[f'sp_m{scales[0]}'] / (new_features[f'sp_m{scales[-1]}'] + 1e-6)

    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    return X

def add_state_features(X, center_x, center_y, fps):
    """Behavioral state transitions; bins adjusted so semantics are fps-invariant."""
    speed = safe_sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)  # cm/s
    w_ma = _scale(15, fps)
    speed_ma = speed.rolling(w_ma, min_periods=max(1, w_ma // 3)).mean()

    try:
        # Original bins (cm/frame): [-inf, 0.5, 2.0, 5.0, inf]
        # Convert to cm/s by multiplying by fps to keep thresholds consistent across fps.
        bins = [-np.inf, 0.5 * fps, 2.0 * fps, 5.0 * fps, np.inf]
        speed_states = pd.cut(speed_ma, bins=bins, labels=[0, 1, 2, 3]).astype(float)

        new_features = {}
        for window in [60, 120]:
            ws = _scale(window, fps)
            if len(speed_states) >= ws:
                for state in [0, 1, 2, 3]:
                    new_features[f's{state}_{window}'] = (
                        (speed_states == state).astype(float)
                        .rolling(ws, min_periods=max(1, ws // 6)).mean()
                    )
                state_changes = (speed_states != speed_states.shift(1)).astype(float)
                new_features[f'trans_{window}'] = state_changes.rolling(ws, min_periods=max(1, ws // 6)).sum()
        
        if new_features:
            X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)
    except Exception:
        pass

    return X

def add_longrange_features(X, center_x, center_y, fps):
    """Long-range temporal features (windows & spans scaled by fps)."""
    new_features = {}
    for window in [120, 240]:
        ws = _scale(window, fps)
        if len(center_x) >= ws:
            new_features[f'x_ml{window}'] = center_x.rolling(ws, min_periods=max(5, ws // 6)).mean()
            new_features[f'y_ml{window}'] = center_y.rolling(ws, min_periods=max(5, ws // 6)).mean()

    # EWM spans also interpreted in frames
    for span in [60, 120]:
        s = _scale(span, fps)
        new_features[f'x_e{span}'] = center_x.ewm(span=s, min_periods=1).mean()
        new_features[f'y_e{span}'] = center_y.ewm(span=s, min_periods=1).mean()

    speed = safe_sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)  # cm/s
    for window in [60, 120]:
        ws = _scale(window, fps)
        if len(speed) >= ws:
            new_features[f'sp_pct{window}'] = speed.rolling(ws, min_periods=max(5, ws // 6)).rank(pct=True)

    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    return X

def add_interaction_features(X, mouse_pair, avail_A, avail_B, fps):
    """Social interaction features (windows scaled by fps)."""
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X

    rel_x = mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x']
    rel_y = mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y']
    rel_dist = safe_sqrt(rel_x**2 + rel_y**2)

    # per-frame velocities (cm/frame)
    A_vx = mouse_pair['A']['body_center']['x'].diff()
    A_vy = mouse_pair['A']['body_center']['y'].diff()
    B_vx = mouse_pair['B']['body_center']['x'].diff()
    B_vy = mouse_pair['B']['body_center']['y'].diff()

    A_lead = (A_vx * rel_x + A_vy * rel_y) / (safe_sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (safe_sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)

    new_features = {}
    for window in [30, 60]:
        ws = _scale(window, fps)
        new_features[f'A_ld{window}'] = A_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()
        new_features[f'B_ld{window}'] = B_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()

    approach = -rel_dist.diff()  # decreasing distance => positive approach
    chase = approach * B_lead
    w = 30
    ws = _scale(w, fps)
    new_features[f'chase_{w}'] = chase.rolling(ws, min_periods=max(1, ws // 6)).mean()

    for window in [60, 120]:
        ws = _scale(window, fps)
        A_sp = safe_sqrt(A_vx**2 + A_vy**2)
        B_sp = safe_sqrt(B_vx**2 + B_vy**2)
        new_features[f'sp_cor{window}'] = A_sp.rolling(ws, min_periods=max(1, ws // 6)).corr(B_sp)

    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    return X

def add_fft_features(X, signal, signal_name, fps, window_size=120):
    """
    Extract FFT-based frequency domain features using Vectorized Spectrogram.
    Massively faster than rolling loop.
    """
    from scipy.signal import spectrogram
    
    ws = _scale(window_size, fps)
    N = len(signal)
    
    if N < ws:
        return X
    
    try:
        # Fill NaNs for FFT input
        sig_filled = signal.ffill().bfill().fillna(0).values
        
        # Compute Spectrogram
        # nperseg=ws, noverlap=ws-1 gives stride=1, mimicking rolling(window=ws)
        f, t, Sxx = spectrogram(sig_filled, fs=fps, window='hann', 
                                nperseg=ws, noverlap=ws-1, 
                                mode='psd', detrend='constant', scaling='density')
        
        # Sxx shape: (n_freqs, n_time_steps)
        # Transpose to (n_time_steps, n_freqs) to align with time
        Sxx = Sxx.T + 1e-20 # Avoid log(0)
        
        # 1. Spectral Energy
        spectral_energy = np.sum(Sxx, axis=1)
        
        # 2. Spectral Entropy
        # Normalize to probability distribution along freq axis
        P_norm = Sxx / (spectral_energy[:, None])
        spectral_entropy = -np.sum(P_norm * np.log2(P_norm), axis=1)
        
        # 3. Dominant Frequency
        # Skip DC component (index 0)
        dom_idx = np.argmax(Sxx[:, 1:], axis=1) + 1
        dominant_freq = f[dom_idx]
        
        # 4. Low Band Power (0-1 Hz)
        mask_low = (f >= 0) & (f < 1.0)
        power_low = np.sum(Sxx[:, mask_low], axis=1)
        
        # 5. High Band Power (1-5 Hz)
        mask_high = (f >= 1.0) & (f < 5.0)
        power_high = np.sum(Sxx[:, mask_high], axis=1)
        
        # Padding to match original length N
        # The spectrogram output length is roughly N - ws + 1
        pad_head = ws // 2
        
        feats_dict = {
            f'{signal_name}_fft_domfreq': dominant_freq,
            f'{signal_name}_fft_energy': spectral_energy,
            f'{signal_name}_fft_entropy': spectral_entropy,
            f'{signal_name}_fft_power_low': power_low,
            f'{signal_name}_fft_power_high': power_high
        }
        
        new_features = {}
        valid_len = len(spectral_energy)
        
        for name, val_array in feats_dict.items():
            arr = np.full(N, np.nan)
            # Ensure we don't overflow if calculation is slightly off
            end_idx = min(N, pad_head + valid_len)
            arr[pad_head : end_idx] = val_array[:end_idx-pad_head]
            new_features[name] = arr
            
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)
        
    except Exception as e:
        if verbose: print(f"FFT feature extraction failed: {e}")
    
    return X
def add_advanced_kinematics(X, center_x, center_y, fps):
    """Add acceleration, jerk, and angular velocity features."""
    # Velocity
    vel_x = center_x.diff() * fps  # cm/s
    vel_y = center_y.diff() * fps
    speed = safe_sqrt(vel_x**2 + vel_y**2)
    
    # Acceleration (cm/s^2)
    acc_x = vel_x.diff() * fps
    acc_y = vel_y.diff() * fps
    acc_mag = safe_sqrt(acc_x**2 + acc_y**2)
    
    # Jerk (cm/s^3)
    jerk_x = acc_x.diff() * fps
    jerk_y = acc_y.diff() * fps
    jerk_mag = safe_sqrt(jerk_x**2 + jerk_y**2)
    
    # Direction angle
    direction = np.arctan2(vel_y, vel_x)
    
    # Angular velocity (rad/s)
    angular_vel = direction.diff() * fps
    # Unwrap to handle -pi to pi discontinuities
    angular_vel = np.where(angular_vel > np.pi, angular_vel - 2*np.pi, angular_vel)
    angular_vel = np.where(angular_vel < -np.pi, angular_vel + 2*np.pi, angular_vel)
    
    # Add raw features
    new_features = {}
    new_features['speed'] = speed
    new_features['acc_mag'] = acc_mag
    new_features['jerk_mag'] = jerk_mag
    new_features['ang_vel'] = angular_vel
    
    # Add rolling statistics
    for w in [15, 30]:
        ws = _scale(w, fps)
        roll = dict(window=ws, min_periods=max(1, ws // 4))
        new_features[f'acc_mean_{w}'] = acc_mag.rolling(**roll).mean()
        new_features[f'acc_std_{w}'] = acc_mag.rolling(**roll).var().clip(lower=0).pow(0.5)
        new_features[f'jerk_mean_{w}'] = jerk_mag.rolling(**roll).mean()
        new_features[f'ang_vel_std_{w}'] = pd.Series(angular_vel).rolling(**roll).var().clip(lower=0).pow(0.5)
    
    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    return X

def add_body_angle_features(X, nose_x, nose_y, body_x, body_y, tail_x, tail_y, fps):
    """Add head/body angle features relative to arena."""
    # Head direction (nose relative to body center)
    head_dir_x = nose_x - body_x
    head_dir_y = nose_y - body_y
    head_angle = np.arctan2(head_dir_y, head_dir_x)  # angle relative to arena
    
    # Body direction (body center relative to tail)
    body_dir_x = body_x - tail_x
    body_dir_y = body_y - tail_y
    body_angle = np.arctan2(body_dir_y, body_dir_x)
    
    # Angular velocities
    head_ang_vel = head_angle.diff() * fps
    body_ang_vel = body_angle.diff() * fps
    
    # Unwrap discontinuities
    head_ang_vel = np.where(head_ang_vel > np.pi, head_ang_vel - 2*np.pi, head_ang_vel)
    head_ang_vel = np.where(head_ang_vel < -np.pi, head_ang_vel + 2*np.pi, head_ang_vel)
    body_ang_vel = np.where(body_ang_vel > np.pi, body_ang_vel - 2*np.pi, body_ang_vel)
    body_ang_vel = np.where(body_ang_vel < -np.pi, body_ang_vel + 2*np.pi, body_ang_vel)
    
    new_features = {}
    new_features['head_angle'] = head_angle
    new_features['body_angle'] = body_angle
    new_features['head_ang_vel'] = head_ang_vel
    new_features['body_ang_vel'] = body_ang_vel
    
    # Rolling statistics
    for w in [15, 30]:
        ws = _scale(w, fps)
        new_features[f'head_ang_std_{w}'] = pd.Series(head_angle).rolling(ws, min_periods=max(1, ws//4)).var().clip(lower=0).pow(0.5)
        new_features[f'head_ang_vel_mean_{w}'] = pd.Series(head_ang_vel).rolling(ws, min_periods=max(1, ws//4)).mean()
    
    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    return X

def add_enhanced_social_features(X, mouse_pair, avail_A, avail_B, fps):
    """Add approach angle and facing target features."""
    if not all(p in avail_A for p in ['nose', 'tail_base', 'body_center']):
        return X
    if not all(p in avail_B for p in ['nose', 'tail_base', 'body_center']):
        return X
    
    # Relative position
    rel_x = mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x']
    rel_y = mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y']
    
    # A's heading direction
    A_head_x = mouse_pair['A']['nose']['x'] - mouse_pair['A']['tail_base']['x']
    A_head_y = mouse_pair['A']['nose']['y'] - mouse_pair['A']['tail_base']['y']
    A_heading = np.arctan2(A_head_y, A_head_x)
    
    # Angle from A to B
    angle_to_B = np.arctan2(rel_y, rel_x)
    
    # Approach angle: difference between heading and direction to other mouse
    approach_angle = A_heading - angle_to_B
    # Normalize to [-pi, pi]
    approach_angle = approach_angle.replace([np.inf, -np.inf], np.nan)
    approach_angle = np.arctan2(np.sin(approach_angle), np.cos(approach_angle))
    
    new_features = {}
    new_features['approach_angle'] = approach_angle
    
    # Binary facing indicator (facing if angle < 45 degrees)
    new_features['facing_target'] = (np.abs(approach_angle.fillna(np.pi)) < np.pi/4).astype(float)    
    # Continuous facing feature (Cosine)
    new_features['facing_cosine'] = np.cos(approach_angle)
    
    # Approach speed (Projected Velocity of A towards B)
    # 1. Vector from A to B
    vec_AB_x = mouse_pair['B']['body_center']['x'] - mouse_pair['A']['body_center']['x']
    vec_AB_y = mouse_pair['B']['body_center']['y'] - mouse_pair['A']['body_center']['y']
    dist_AB = safe_sqrt(vec_AB_x**2 + vec_AB_y**2) + 1e-6

    # 2. Velocity of A
    vel_A_x = mouse_pair['A']['body_center']['x'].diff().fillna(0)
    vel_A_y = mouse_pair['A']['body_center']['y'].diff().fillna(0)

    # 3. Projected velocity
    new_features['approach_speed'] = (vel_A_x * vec_AB_x + vel_A_y * vec_AB_y) / dist_AB * fps

    
    # Rolling statistics
    ws = _scale(30, fps)
    new_features['facing_pct_30'] = new_features['facing_target'].rolling(ws, min_periods=max(1, ws//6)).mean()
    
    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    return X


In [ ]:
def add_egocentric_features(X, mouse_df, fps):
    """
    Biến đổi tọa độ sang hệ quy chiếu lấy chuột làm tâm (Egocentric).
    Chuẩn hóa sao cho: Body Center tại (0,0), Mũi hướng về phía dương trục X.
    """
    # 1. Xác định trục cơ thể (Spine Vector)
    if not all(p in mouse_df.columns.get_level_values(0) for p in ['nose', 'body_center']):
        return X

    # Vector từ tâm đến mũi
    dx = mouse_df['nose']['x'] - mouse_df['body_center']['x']
    dy = mouse_df['nose']['y'] - mouse_df['body_center']['y']
    
    # Góc quay của chuột so với trục hoành của camera
    angle = np.arctan2(dy, dx)
    cos_a = np.cos(-angle)
    sin_a = np.sin(-angle)

    new_feats = {}
    
    # 2. Xoay tọa độ các bộ phận quan trọng
    # Chỉ quan tâm các bộ phận chính để giảm chiều dữ liệu
    key_parts = ['ear_left', 'ear_right', 'tail_base', 'tail_tip']
    available_parts = mouse_df.columns.get_level_values(0)

    for part in key_parts:
        if part in available_parts:
            # Tọa độ tương đối so với tâm
            rx = mouse_df[part]['x'] - mouse_df['body_center']['x']
            ry = mouse_df[part]['y'] - mouse_df['body_center']['y']
            
            # Phép quay ma trận 2D
            # x_new = x*cos - y*sin
            # y_new = x*sin + y*cos
            x_rot = rx * cos_a - ry * sin_a
            y_rot = rx * sin_a + ry * cos_a
            
            new_feats[f'ego_x_{part}'] = x_rot
            new_feats[f'ego_y_{part}'] = y_rot

    if new_feats:
        X = pd.concat([X, pd.DataFrame(new_feats, index=X.index)], axis=1)
        
    return X

def add_grooming_features(X, mouse_df, fps):
    """
    Phát hiện hành vi chải chuốt: Thân đứng yên nhưng đầu di chuyển/rung lắc.
    """
    if not all(p in mouse_df.columns.get_level_values(0) for p in ['nose', 'body_center']):
        return X

    # Tốc độ mũi
    nose_speed = np.sqrt(mouse_df['nose']['x'].diff()**2 + mouse_df['nose']['y'].diff()**2) * fps
    # Tốc độ thân
    body_speed = np.sqrt(mouse_df['body_center']['x'].diff()**2 + mouse_df['body_center']['y'].diff()**2) * fps

    # Tỷ lệ tách biệt (Decoupling Ratio)
    # Thêm 1e-3 để tránh chia cho 0
    decouple = nose_speed / (body_speed + 1e-3)
    
    # Làm mượt (Smoothing) vì hành vi này thường kéo dài ít nhất 0.5s
    w = int(0.5 * fps)
    
    new_feats = {}
    new_feats['head_body_ratio'] = decouple.rolling(w).median()
    new_feats['body_immobile'] = (body_speed < 2.0).astype(float) # Thân di chuyển dưới 2cm/s
    new_feats['nose_active'] = (nose_speed > 5.0).astype(float)   # Mũi di chuyển trên 5cm/s
    
    # Kết hợp logic: Grooming = Thân tĩnh AND Mũi động
    new_feats['grooming_score'] = new_feats['body_immobile'] * new_feats['nose_active']

    X = pd.concat([X, pd.DataFrame(new_feats, index=X.index)], axis=1)
    return X

def add_temporal_asymmetry(X, center_x, center_y, fps):
    """
    So sánh vận tốc tương lai và quá khứ để phát hiện chuyển đổi trạng thái (Attack onset/offset).
    """
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * fps
    
    # Cửa sổ 1 giây
    w = int(1.0 * fps)
    
    # Vận tốc trung bình Quá khứ (shift dương)
    # min_periods=1 để tránh NaN ở đầu video
    past_mean = speed.rolling(window=w, min_periods=1).mean()
    
    # Vận tốc trung bình Tương lai (shift ngược bằng cách đảo ngược chuỗi)
    # Hoặc dùng: speed.shift(-w).rolling(w).mean() nhưng cách dưới chính xác hơn cho biên
    future_mean = speed.iloc[::-1].rolling(window=w, min_periods=1).mean().iloc[::-1]
    
    new_feats = {}
    # Delta V: Dương -> Đang tăng tốc (Attack start), Âm -> Đang giảm tốc (Stop)
    new_feats['accel_trend_1s'] = future_mean - past_mean
    
    # Ratio: Thay đổi gấp bao nhiêu lần
    new_feats['accel_ratio_1s'] = future_mean / (past_mean + 1e-3)

    X = pd.concat([X, pd.DataFrame(new_feats, index=X.index)], axis=1)
    return X

In [ ]:
drop_body_parts =  ['headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
                    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
                    'spine_1', 'spine_2',
                    'tail_middle_1', 'tail_middle_2', 'tail_midpoint']

def generate_mouse_data(dataset, traintest, traintest_directory=None, generate_single=True, generate_pair=True):
    if traintest_directory is None:
        traintest_directory = f"{CFG.BASE_PATH}/{traintest}_tracking"

    for _, row in dataset.iterrows():
        lab_id = row.lab_id
        if lab_id.startswith('MABe22') and traintest == 'train': continue #MABe22 kh co label
        video_id = row.video_id

        if type(row.behaviors_labeled) != str: # Nếu không có nhãn đánh dấu -> skip
            print('No labeled behaviors:', lab_id, video_id, type(row.behaviors_labeled), row.behaviors_labeled)
            continue

        path = f'{traintest_directory}/{lab_id}/{video_id}.parquet'
        vid = pd.read_parquet(path)

        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@drop_body_parts)")

        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])

        if (pvid.isna().any().any()):
            if verbose and traintest == 'test': print('video with missing values', video_id, traintest, len(vid), 'frames')
        else:
            if verbose and traintest == 'test': print('video with all values', video_id, traintest, len(vid), 'frames')
        del vid

        pvid = pvid.reorder_levels([1,2,0], axis=1).T.sort_index().T
        
        # 1. Đổi đơn vị: pixels -> cm
        pvid /= row.pix_per_cm_approx

        vid_behaviors = json.loads(row.behaviors_labeled)
        vid_behaviors = sorted(list({b.replace("''","") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
    
        if traintest == 'train':
            try:
                annot = pd.read_parquet(path.replace('train_tracking', 'train_annotation'))
            except FileNotFoundError:
                continue
        if generate_single:
            vid_behaviors_subset = vid_behaviors.query("target == 'self'")

            for mouse_id_str in np.unique(vid_behaviors_subset.agent):
                try:
                    mouse_id = int(mouse_id_str[-1])
                    vid_agent_actions = np.unique(vid_behaviors_subset.query("agent == @mouse_id_str").action)

                    single_mouse = pvid.loc[:, mouse_id]
                    assert len(single_mouse) == len(pvid)
                    single_mouse_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': mouse_id_str,
                        'target_id': 'self',
                        'video_frame': single_mouse.index
                    })

                    if traintest == 'train':
                        single_mouse_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=single_mouse.index) 
                        annot_subset = annot.query("(agent_id == @mouse_id) & (target_id == @mouse_id)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            single_mouse_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'single', single_mouse, single_mouse_meta, single_mouse_label
                    else:
                        if verbose: print('- test single', video_id, mouse_id)
                        yield 'single', single_mouse, single_mouse_meta, vid_agent_actions

                except KeyError:
                    pass

        if generate_pair:
            vid_behaviors_subset = vid_behaviors.query("target != 'self'")
            if len(vid_behaviors_subset) > 0:
                for agent, target in itertools.permutations(np.unique(pvid.columns.get_level_values('mouse_id')), 2):
                    agent_str = f'mouse{agent}'
                    target_str = f'mouse{target}'

                    vid_agent_actions = np.unique(vid_behaviors_subset.query("(agent == @agent_str) & (target == @target_str)").action)
                    mouse_pair = pd.concat([pvid[agent], pvid[target]], axis=1, keys=['A', 'B'])
                    assert len(mouse_pair) == len(pvid)
                   
                    mouse_pair_meta = pd.DataFrame({
                        'video_id': video_id,
                        'agent_id': agent_str,
                        'target_id': target_str,
                        'video_frame': mouse_pair.index
                    })
                    
                    if traintest == 'train':
                        mouse_pair_label = pd.DataFrame(0.0, columns=vid_agent_actions, index=mouse_pair.index)
                        annot_subset = annot.query("(agent_id == @agent) & (target_id == @target)")
                        for i in range(len(annot_subset)):
                            annot_row = annot_subset.iloc[i]
                            mouse_pair_label.loc[annot_row['start_frame']:annot_row['stop_frame'], annot_row.action] = 1.0
                        yield 'pair', mouse_pair, mouse_pair_meta, mouse_pair_label
                    else:
                        if verbose: print('- test pair', video_id, agent, target)
                        yield 'pair', mouse_pair, mouse_pair_meta, vid_agent_actions
def transform_single(single_mouse, body_parts_tracked, fps):
    """Transform from cartesian coordinates to distance representation.

    Parameters:
    single_mouse: dataframe with coordinates of the body parts of one mouse
                  shape (n_samples, n_body_parts * 2)
                  two-level MultiIndex on columns
    body_parts_tracked: list of body parts
    """
    available_body_parts = single_mouse.columns.get_level_values(0)
    
    # Smooth coordinates to reduce noise
    single_mouse = smooth_coordinates(single_mouse, sigma=1.5)
    
    # X là toàn bộ khoảng cách giữa các bộ phận của con chuột
    X = pd.DataFrame({
            f"{part1}+{part2}": np.square(single_mouse[part1] - single_mouse[part2]).sum(axis=1, skipna=False)
            for part1, part2 in itertools.combinations(body_parts_tracked, 2) if part1 in available_body_parts and part2 in available_body_parts
        })
    X = X.reindex(columns=[f"{part1}+{part2}" for part1, part2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    if all(p in single_mouse.columns for p in ['ear_left', 'ear_right', 'tail_base']):
        # lag ~ 10 frame trong 30fps
        lag = _scale(10, fps) 
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag) 
        X = pd.concat([
            X, 
            pd.DataFrame({
                'speed_left': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
                'speed_right': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
                'speed_left2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
                'speed_right2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
            })
        ], axis=1)

    new_features = {}
    # Elongation: độ kéo dài của cơ thể
    if 'nose+tail_base' in X.columns and "ear_left+ear_right" in X.columns:
        new_features['elong'] = safe_sqrt(X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6))

    # Góc từ mũi -> thân -> đuôi
    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose']-single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        new_features['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (safe_sqrt(v1['x']**2 + v1['y']**2) * safe_sqrt(v2['x']**2 + v2['y']**2))

    # Khoảng cách giữa mũi và đuôi
    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = safe_sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                          (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            new_features[f'nt_lg{lag}'] = nt_dist.shift(l) # khoảng cách giữa nose-tail trong quá khứ
            new_features[f'nt_df{lag}'] = nt_dist - nt_dist.shift(l) # Độ thay đổi so với hiện tại

    # Rolling statistic dựa trên body center
    if 'body_center' in available_body_parts:
        center_x = single_mouse['body_center']['x']
        center_y = single_mouse['body_center']['y']

        for w in [5, 15, 30, 60]:
            w_scale = _scale(w, fps)
            roll = dict(window=w_scale, min_periods=1, center=True)
            new_features[f'cx_mean_{w}'] = center_x.rolling(**roll).mean()
            new_features[f'cy_mean_{w}'] = center_y.rolling(**roll).mean()
            new_features[f'cx_std_{w}'] = center_x.rolling(**roll).var().clip(lower=0).pow(0.5)
            new_features[f'cy_std_{w}'] = center_y.rolling(**roll).var().clip(lower=0).pow(0.5)
            new_features[f'cx_range_{w}'] = center_x.rolling(**roll).max() - center_x.rolling(**roll).min()
            new_features[f'cy_range_{w}'] = center_y.rolling(**roll).max() - center_y.rolling(**roll).min()
            new_features[f'variablitiy_{w}'] = safe_sqrt(center_x.diff().rolling(w_scale, min_periods=1).var().clip(lower=0) + 
                                             center_y.diff().rolling(w_scale, min_periods=1).var().clip(lower=0))
            new_features[f'displacement_{w}'] = safe_sqrt(center_x.diff().rolling(w_scale, min_periods=1).sum()**2 + 
                                              center_y.diff().rolling(w_scale, min_periods=1).sum()**2)
            
    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    # Call helper functions (they now use pd.concat internally)
    if 'body_center' in available_body_parts:
        X = add_curvature_features(X, center_x, center_y, fps)
        X = add_multiscale_features(X, center_x, center_y, fps)
        X = add_state_features(X, center_x, center_y, fps)
        X = add_longrange_features(X, center_x, center_y, fps)
        
        # New features: Advanced kinematics
        X = add_advanced_kinematics(X, center_x, center_y, fps)
        
        # New features: FFT on speed signal
        speed_signal = safe_sqrt(center_x.diff()**2 + center_y.diff()**2)
        X = add_fft_features(X, speed_signal, 'speed', fps, window_size=120)
        
        # New features: Body angles (if body parts available)
        if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
            X = add_body_angle_features(X, 
                                       single_mouse['nose']['x'], single_mouse['nose']['y'],
                                       center_x, center_y,
                                       single_mouse['tail_base']['x'], single_mouse['tail_base']['y'],
                                       fps)

    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_dist = safe_sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 + 
                           (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        
        new_features_ear = {}
        for offset in [-30, -20, -10, 10, 20, 30]:
            o = _scale_signed(offset, fps)
            new_features_ear[f'ear_dist_o{offset}'] = ear_dist.shift(-o)
        
        w = _scale(30, fps)
        new_features_ear['ear_consistency'] = ear_dist.rolling(w, min_periods=1, center=True).var().clip(lower=0).pow(0.5) / (ear_dist.rolling(w, min_periods=1, center=True).mean() + 1e-6)
        
        if new_features_ear:
            X = pd.concat([X, pd.DataFrame(new_features_ear, index=X.index)], axis=1)

# --- ĐOẠN CODE THÊM MỚI ---
    # 1. Thêm Egocentric (Quan trọng nhất)
    X = add_egocentric_features(X, single_mouse, fps)
    
    # 2. Thêm Grooming Features (Cải thiện lớp 'Grooming' và 'Other')
    X = add_grooming_features(X, single_mouse, fps)
    
    # 3. Thêm Temporal Asymmetry (Cải thiện lớp 'Attack' và 'Chase')
    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']
        X = add_temporal_asymmetry(X, cx, cy, fps)
    # --------------------------
    return X.astype(np.float32, copy=False)

def transform_pair(mouse_pair, body_parts_tracked, fps):
    """Transform from cartesian coordinates to distance representation.

    Parameters:
    mouse_pair: dataframe with coordinates of the body parts of two mice
                  shape (n_samples, 2 * n_body_parts * 2)
                  three-level MultiIndex on columns
    body_parts_tracked: list of body parts
    """
    # drop_body_parts =  ['ear_left', 'ear_right',
    #                     'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
    #                     'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
    #                     'tail_midpoint']
    # if len(body_parts_tracked) > 5:
    #     body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
    available_body_parts_A = mouse_pair['A'].columns.get_level_values(0)
    available_body_parts_B = mouse_pair['B'].columns.get_level_values(0)
    
    # Smooth coordinates to reduce noise
    mouse_pair['A'] = smooth_coordinates(mouse_pair['A'], sigma=1.5)
    mouse_pair['B'] = smooth_coordinates(mouse_pair['B'], sigma=1.5)
    
    ETHOLOGICAL_PAIRS = [
        ('nose', 'nose'),
        ('nose', 'tail_base'),
        ('tail_base', 'nose'),
        ('nose', 'body_center'),
        ('body_center', 'nose'),
        ('body_center', 'body_center'),
        ('tail_base', 'tail_base')
    ]
    
    X = pd.DataFrame({
            f"12+{part1}+{part2}": np.square(mouse_pair['A'][part1] - mouse_pair['B'][part2]).sum(axis=1, skipna=False)
            for part1, part2 in ETHOLOGICAL_PAIRS if part1 in available_body_parts_A and part2 in available_body_parts_B
        })
    X = X.reindex(columns=[f"12+{part1}+{part2}" for part1, part2 in ETHOLOGICAL_PAIRS], copy=False)

    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps)
        shifted_A = mouse_pair['A']['ear_left'].shift(lag)
        shifted_B = mouse_pair['B']['ear_left'].shift(lag)
        X = pd.concat([
            X,
            pd.DataFrame({
                'speed_left_A': np.square(mouse_pair['A']['ear_left'] - shifted_A).sum(axis=1, skipna=False),
                'speed_left_AB': np.square(mouse_pair['A']['ear_left'] - shifted_B).sum(axis=1, skipna=False),
                'speed_left_B': np.square(mouse_pair['B']['ear_left'] - shifted_B).sum(axis=1, skipna=False),
            })
        ], axis=1)

    new_features = {}
    # góc giữa 2 con chuột
    if all(p in available_body_parts_A for p in ['nose', 'tail_base']) and all(p in available_body_parts_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        new_features['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            safe_sqrt(dir_A['x']**2 + dir_A['y']**2) * safe_sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)
        
    # Khoảng cách giữa 2 mũi của 2 con chuột, % chúng gần
    if all(p in available_body_parts_A for p in ['nose']) and all(p in available_body_parts_B for p in ['nose']):
        nose_nose = safe_sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                     (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)
        for lag in [10, 20, 40]:
            lag_ = _scale(lag, fps)
            new_features[f'nose-nose_lag_{lag}'] = nose_nose.shift(lag_) # Khoảng cách của mũi chúng theo lag
            new_features[f'nose-nose_change_{lag}'] = nose_nose - nose_nose.shift(lag_) # Sự thay đổi khoảng cách của chúng
            is_close = nose_nose.fillna(np.inf).lt(10).astype(float)
            new_features[f'close_percentage_{lag}'] = is_close.rolling(lag_, min_periods=1).mean() # Trong _lag frame thì có bao nhiêu % là mũi của chúng gần
    
    # if 'body_center' in available_body_parts_A and 'body_center' in available_body_parts_B:
    #     cd = safe_sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
    #                  (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
    #     cd = cd.fillna(np.inf)
    #     X['very_close'] = (cd < 5.0).astype(float)
    #     X['close']   = ((cd >= 5.0) & (cd < 15.0)).astype(float)
    #     X['med']   = ((cd >= 15.0) & (cd < 30.0)).astype(float)
    #     X['far']   = (cd >= 30.0).astype(float)

    # Thống kê dựa trên khoảng cách của body center
    if 'body_center' in available_body_parts_A and 'body_center' in available_body_parts_B:
        center_d = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)
        Avx = mouse_pair['A']['body_center']['x'].diff()
        Avy = mouse_pair['A']['body_center']['y'].diff()
        Bvx = mouse_pair['B']['body_center']['x'].diff()
        Bvy = mouse_pair['B']['body_center']['y'].diff()
        coord = Avx * Bvx + Avy * Bvy
        val = (Avx * Bvx + Avy * Bvy) / (safe_sqrt(Avx**2 + Avy**2) * safe_sqrt(Bvx**2 + Bvy**2) + 1e-6)

        for w in [5, 15, 30, 60]:
            w_scale = _scale(w, fps)
            roll = dict(window=w_scale, min_periods=1, center=True)
            new_features[f'd_mean_{w}'] = center_d.rolling(**roll).mean()
            new_features[f'd_std_{w}'] = center_d.rolling(**roll).var().clip(lower=0).pow(0.5)
            new_features[f'd_max_{w}'] = center_d.rolling(**roll).max()
            new_features[f'd_min_{w}'] = center_d.rolling(**roll).min()
            d_var = center_d.rolling(**roll).var()
            new_features[f'interaction_{w}'] = 1 / (1 + d_var) # neu var thap -> khoang cach on dinh -> int cao
            new_features[f'co_m{w}'] = coord.rolling(**roll).mean()
            new_features[f'co_s{w}'] = coord.rolling(**roll).var().clip(lower=0).pow(0.5)
        
        for off in [-30, -20, -10, 0, 10, 20, 30]:
            o = _scale_signed(off, fps)
            new_features[f'va_{off}'] = val.shift(-o)
        ws = _scale(30, fps)
        new_features['int_con'] = center_d.rolling(ws, min_periods=1, center=True).var().clip(lower=0).pow(0.5) / \
                       (center_d.rolling(ws, min_periods=1, center=True).mean() + 1e-6)
        
    if new_features:
        X = pd.concat([X, pd.DataFrame(new_features, index=X.index)], axis=1)

    if 'body_center' in available_body_parts_A and 'body_center' in available_body_parts_B:
        X = add_interaction_features(X, mouse_pair, available_body_parts_A, available_body_parts_B, fps)
        
        # New features: Enhanced social interaction features
        X = add_enhanced_social_features(X, mouse_pair, available_body_parts_A, available_body_parts_B, fps)
        
        # New features: FFT on inter-mouse distance
        if 'body_center' in available_body_parts_A and 'body_center' in available_body_parts_B:
            inter_dist = safe_sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                                (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
            X = add_fft_features(X, inter_dist, 'dist', fps, window_size=120)

    # Replace inf values with 0
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    # Replace inf values with 0
    X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
    return X.astype(np.float32, copy=False)


In [ ]:
class DataSampler(ClassifierMixin, BaseEstimator):
    def __init__(self, estimator, neg_pos_ratio=10.0): 
        self.estimator = estimator
        self.neg_pos_ratio = neg_pos_ratio

    def fit(self, X, y):
        X_arr = np.array(X, copy=False)
        y_arr = np.array(y, copy=False)
        
        pos_indices = np.where(y_arr == 1)[0]
        neg_indices = np.where(y_arr == 0)[0]
        
        if len(pos_indices) == 0:
            self.estimator.fit(X_arr[::10], y_arr[::10])
            self.classes_ = self.estimator.classes_
            return self

        n_neg_keep = int(len(pos_indices) * self.neg_pos_ratio)
        
        if len(neg_indices) > n_neg_keep:
            kept_neg_indices = np.random.choice(neg_indices, n_neg_keep, replace=False)
        else:
            kept_neg_indices = neg_indices
            
        final_indices = np.concatenate([pos_indices, kept_neg_indices])
        np.random.shuffle(final_indices)
        
        self.estimator.fit(X_arr[final_indices], y_arr[final_indices])
        self.classes_ = self.estimator.classes_
        return self

    def predict_proba(self, X):
        return self.estimator.predict_proba(np.array(X))

    def predict(self, X):
        return self.estimator.predict(np.array(X))


def submit(body_parts_tracked_str, switch_tr, X_tr, label, meta):
    """Produce a submission file for the selected subset of the test data.

    Parameters
    ----------
    body_parts_tracked_str: subset of body parts for filtering the test set
    switch_tr: 'single' or 'pair'
    binary_classifier: classifier with predict_proba
    X_tr: training features as 2d array-like of shape (n_samples, n_features)
    label: dataframe with binary targets (one column per action, may have missing values), index doesn't matter
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame'], index doesn't matter
    
    Output
    ------
    appends to submission_list
    
    """
    # Fit a binary classifier for every action
    model_list = [] # will get a model per action
    for action in label.columns:
        # Filter for samples (video frames) with a defined target (i.e., target is not nan)
        action_mask = ~ label[action].isna().values
        y_action = label[action][action_mask].values.astype(int)

        if not (y_action == 0).all():
            # Train XGBoost
            model_xgb = DataSampler(clone(CFG.model), neg_pos_ratio=10.0)
            model_xgb.fit(X_tr[action_mask], y_action)
            
            # Train CatBoost
            cat_model = CatBoostClassifier(**CAT_PARAMS)
            model_cat = DataSampler(cat_model, neg_pos_ratio=10.0)
            model_cat.fit(X_tr[action_mask], y_action)

            # Feature Importance Logging
            try:
                # XGBoost
                importances_xgb = model_xgb.estimator.feature_importances_
                feature_names = X_tr.columns
                feature_importance_df_xgb = pd.DataFrame({'Feature': feature_names, 'Importance': importances_xgb})
                top_features_xgb = feature_importance_df_xgb.sort_values(by='Importance', ascending=False).head(20)
                print(f"\nTop 20 Features for Action (XGBoost): {action}")
                for index, row in top_features_xgb.iterrows():
                    print(f"  {row['Feature']}: {row['Importance']:.4f}")
                
                # CatBoost
                importances_cat = model_cat.estimator.feature_importances_
                feature_importance_df_cat = pd.DataFrame({'Feature': feature_names, 'Importance': importances_cat})
                top_features_cat = feature_importance_df_cat.sort_values(by='Importance', ascending=False).head(20)
                print(f"\nTop 20 Features for Action (CatBoost): {action}")
                for index, row in top_features_cat.iterrows():
                    print(f"  {row['Feature']}: {row['Importance']:.4f}")

            except Exception as e:
                if verbose: print(f"Could not print feature importance for {action}: {e}")
            
            model_list.append((action, model_xgb, model_cat))

    # Compute test predictions in batches
    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
    if validate_or_submit == 'submit':
        test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
        generator = generate_mouse_data(test_subset, 'test',
                                        generate_single=(switch_tr == 'single'), 
                                        generate_pair=(switch_tr == 'pair'))
        
        fps_lookup = (
            test_subset[['video_id', 'frames_per_second']]
            .drop_duplicates('video_id')
            .set_index('video_id')['frames_per_second']
            .to_dict()
        )

        for switch_te, data_te, meta_te, actions_te in generator:
            assert switch_te == switch_tr
        try:
            # Transform from coordinate representation into distance representation
            fps_i = _fps_from_meta(meta_te, fps_lookup)

            if switch_te == 'single':
                X_te = transform_single(data_te, body_parts_tracked, fps_i) 
            else:
                X_te = transform_pair(data_te, body_parts_tracked, fps_i) 
            if verbose and len(X_te) == 0: print("ERROR: X_te is empty")
            del data_te
    
            # Compute binary predictions
            pred = pd.DataFrame(index=meta_te.video_frame) # will get a column per action
            for action, model_xgb, model_cat in model_list:
                if action in actions_te:
                    p_xgb = model_xgb.predict_proba(X_te)[:, 1]
                    p_cat = model_cat.predict_proba(X_te)[:, 1]
                    pred[action] = (p_xgb + p_cat) / 2.0
            del X_te
            # Probability Smoothing
            ws = _scale(15, fps_i)
            pred = pred.rolling(window=ws, min_periods=1, center=True).mean()

            # Compute multiclass predictions
            
            if pred.shape[1] != 0:
                submission_part = predict_multiclass(pred, meta_te)
                submission_list.append(submission_part)
            else: # this happens if there was no useful training data for the test actions
                if verbose: print(f"  ERROR: no useful training data")
        except KeyError:
            if verbose: print(f'  ERROR: KeyError because of missing bodypart ({switch_tr})')
            del data_te


def predict_multiclass(pred, meta):
    """Derive multiclass predictions from a set of binary predictions.
    
    Parameters
    pred: dataframe of predicted binary probabilities, shape (n_samples, n_actions), index doesn't matter
    meta: dataframe with columns ['video_id', 'agent_id', 'target_id', 'video_frame'], index doesn't matter
    """
    # Default threshold
    default_thresh = 0.3
    
    # Gap fill: khoảng 0.3 giây (~10 frames ở 30fps)
    # Min duration: khoảng 0.1 giây (~3-5 frames)
    GAP_FILL_SIZE = 10
    MIN_DURATION_SIZE = 5 
    
    # 1. Apply thresholds to create a mask
    binary_pred = pd.DataFrame(0, index=pred.index, columns=pred.columns)
    
    for action in pred.columns:
        binary_pred[action] = (pred[action] >= default_thresh).astype(int)

    # --- BẮT ĐẦU ĐOẠN POST-PROCESSING MỚI ---
    # Áp dụng cho từng cột hành động riêng biệt
    for col in binary_pred.columns:
        mask = binary_pred[col].values
        
        # Bước A: Gap Filling (Lấp lỗ hổng)
        # Dùng binary_closing: Nối các đoạn 1 bị đứt quãng bởi các số 0 ngắn
        structure_gap = np.ones(GAP_FILL_SIZE)
        mask_filled = binary_closing(mask, structure=structure_gap).astype(int)
        
        # Bước B: Min Duration Filtering (Lọc nhiễu ngắn)
        # Dùng binary_opening: Xóa các đoạn 1 ngắn hơn kích thước structure
        structure_min = np.ones(MIN_DURATION_SIZE)
        mask_clean = binary_opening(mask_filled, structure=structure_min).astype(int)
        
        # Cập nhật lại cột
        binary_pred[col] = mask_clean
    # --- KẾT THÚC ĐOẠN POST-PROCESSING ---

    # 2. Mask the probabilities
    # Only keep probabilities for actions that passed the threshold
    masked_pred = pred * binary_pred
    
    # 3. Find the action with the highest probability AMONG those that passed
    # Note: If no action passed, masked_pred is all 0s, argmax might return 0 (first index)
    # but we will filter those out using has_action_mask.
    best_action_indices = np.argmax(masked_pred.values, axis=1)
    
    final_actions = np.full(len(pred), -1)
    
    # Only consider rows with at least one action passing threshold
    has_action_mask = binary_pred.sum(axis=1) > 0
    
    final_actions[has_action_mask] = best_action_indices[has_action_mask]
    
    ama = pd.Series(final_actions, index=meta.video_frame)
    # Keep only start and stop frames
    changes_mask = (ama != ama.shift(1)).values # nếu ama != ama.shift(1) -> đổi hành động
    ama_changes = ama[changes_mask] # Chỉ giữ lại những frame thay đổi hành động
    meta_changes = meta[changes_mask]
    # mask selects the start frames
    mask = ama_changes.values >= 0 # lọc những điểm bắt đầu của các action
    mask[-1] = False  
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': pred.columns[ama_changes[mask].values],
        'start_frame': ama_changes.index[mask],
        'stop_frame': ama_changes.index[1:][mask[:-1]]
    })
    
    # Nếu action kéo đến hết video:
    stop_video_id = meta_changes['video_id'][1:][mask[:-1]].values
    stop_agent_id = meta_changes['agent_id'][1:][mask[:-1]].values
    stop_target_id = meta_changes['target_id'][1:][mask[:-1]].values
    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]
        
        if (video_id != stop_video_id[i]) or (agent_id != stop_agent_id[i]) or (target_id != stop_target_id[i]):
            submission_part.stop_frame.iloc[i] = meta.video_frame.iloc[-1] + 1
            
    return submission_part




In [ ]:
def optimize_thresholds_optuna(y_true_dict, y_prob_dict, n_trials=100):
    """
    Optimize thresholds for multiple actions using Optuna to maximize Macro F1.
    """
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    
    action_names = list(y_true_dict.keys())
    
    def objective(trial):
        f1_scores = []
        for action in action_names:
            y_t = y_true_dict[action]
            y_p = y_prob_dict[action]
            
            # Suggest a float for this action
            thresh = trial.suggest_float(action, 0.1, 0.8)
            
            # Calculate F1
            score = f1_score(y_t, (y_p >= thresh).astype(int), zero_division=0)
            f1_scores.append(score)
            
        # Return Macro F1
        return np.mean(f1_scores)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    
    # Extract best thresholds
    best_thresholds = study.best_params
    
    # Print result
    print(f"Optuna Best Macro F1: {study.best_value:.4f}")
    return best_thresholds

def cross_validate_classifier(X, label, meta):
    """
    Use ensemble of XGBoost + CatBoost for validation (consistent with training).
    """
    y_true_dict = {}
    y_prob_dict = {}
    
    print("  Collecting OOF predictions with ENSEMBLE (XGBoost + CatBoost)...")
    
    for action in label.columns:
        action_mask = ~label[action].isna().values
        X_action = X[action_mask]
        y_action = label[action][action_mask].values.astype(int)
        groups_action = meta.video_id[action_mask]
        
        # Skip if not enough data
        if len(np.unique(groups_action)) < 2 or len(np.unique(y_action)) < 2:
            continue
            
        if not (y_action == 0).all():
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore', category=RuntimeWarning)
                try:
                    cv = StratifiedGroupKFold(n_splits=3)
                    
                    # --- THAY ĐỔI CHÍNH: ENSEMBLE XGBoost + CatBoost ---
                    oof_probs_xgb = np.zeros(len(y_action))
                    oof_probs_cat = np.zeros(len(y_action))
                    
                    for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_action, y_action, groups_action)):
                        X_train, X_val = X_action.iloc[train_idx], X_action.iloc[val_idx]
                        y_train, y_val = y_action[train_idx], y_action[val_idx]
                        
                        # Train XGBoost
                        model_xgb = DataSampler(clone(CFG.model), neg_pos_ratio=10.0)
                        model_xgb.fit(X_train, y_train)
                        oof_probs_xgb[val_idx] = model_xgb.predict_proba(X_val)[:, 1]
                        
                        # Train CatBoost
                        cat_model = CatBoostClassifier(**CAT_PARAMS)
                        model_cat = DataSampler(cat_model, neg_pos_ratio=10.0)
                        model_cat.fit(X_train, y_train)
                        oof_probs_cat[val_idx] = model_cat.predict_proba(X_val)[:, 1]
                        
                        del model_xgb, model_cat
                        gc.collect()
                    
                    # ENSEMBLE: Trung bình của XGBoost và CatBoost
                    oof_probs = (oof_probs_xgb + oof_probs_cat) / 2.0
                    
                    y_true_dict[action] = y_action
                    y_prob_dict[action] = oof_probs
                    # --- KẾT THÚC THAY ĐỔI ---
                    
                except Exception as e:
                    print(f"Error in CV for {action}: {e}")
                    continue

    if not y_true_dict:
        return {}

    # Optimize using Optuna
    print("  Optimizing thresholds with Optuna...")
    try:
        best_thresholds = optimize_thresholds_optuna(y_true_dict, y_prob_dict, n_trials=100)
        return best_thresholds
    except Exception as e:
        print(f"Optuna optimization failed: {e}")
        return {}

In [ ]:
def robustify(submission, dataset, traintest, traintest_directory=None):
    """Ensure that the submission conforms to the three rules"""
    if traintest_directory is None:
        traintest_directory = f"{CFG.BASE_PATH}/{traintest}_tracking"

    # Rule 1: Ensure that start_frame >= stop_frame
    old_submission = submission.copy()
    submission = submission[submission.start_frame < submission.stop_frame]
    if len(submission) != len(old_submission):
        print("ERROR: Dropped frames with start >= stop")
    
    # Rule 2: Avoid multiple predictions for the same frame from one agent/target pair
    old_submission = submission.copy()
    group_list = []
    for _, group in submission.groupby(['video_id', 'agent_id', 'target_id']):
        group = group.sort_values('start_frame')
        mask = np.ones(len(group), dtype=bool)
        last_stop_frame = 0
        for i, (_, row) in enumerate(group.iterrows()):
            if row['start_frame'] < last_stop_frame:
                mask[i] = False
            else:
                last_stop_frame = row['stop_frame']
        group_list.append(group[mask])
    submission = pd.concat(group_list)
    if len(submission) != len(old_submission):
        print("ERROR: Dropped duplicate frames")

    # Rule 3: Submit something for every video
    # Fill missing videos as in https://www.kaggle.com/code/ambrosm/mabe-validated-baseline-without-machine-learning
    s_list = []
    for idx, row in dataset.iterrows():
        lab_id = row['lab_id']
        if lab_id.startswith('MABe22'):
            continue
        video_id = row['video_id']
        if (submission.video_id == video_id).any():
            continue

        if verbose: print(f"Video {video_id} has no predictions.")
        
        # Load video
        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
    
        # Determine the behaviors of this video
        vid_behaviors = eval(row['behaviors_labeled'])
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])
    
        # Determine start_frame and stop_frame
        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1
    
        # Predict all possible actions as often as possible
        for (agent, target), actions in vid_behaviors.groupby(['agent', 'target']):
            batch_length = int(np.ceil((stop_frame - start_frame) / len(actions)))
            for i, (_, action_row) in enumerate(actions.iterrows()):
                batch_start = start_frame + i * batch_length
                batch_stop = min(batch_start + batch_length, stop_frame)
                s_list.append((video_id, agent, target, action_row['action'], batch_start, batch_stop))

    if len(s_list) > 0:
        submission = pd.concat([
            submission,
            pd.DataFrame(s_list, columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        ])
        print("ERROR: Filled empty videos")

    submission = submission.reset_index(drop=True)
    return submission


In [ ]:
train = pd.read_csv(CFG.train_path)
train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)
train_without_mabe22 = train.query("~ lab_id.str.startswith('MABe22_')")
test = pd.read_csv(CFG.test_path)

body_parts_tracked_list = list(np.unique(train.body_parts_tracked))
# %%time
f1_list = []
submission_list = []
for section in range(1, len(body_parts_tracked_list)): # skip index 0 (MABe22)
    body_parts_tracked_str = body_parts_tracked_list[section]
    try:
        body_parts_tracked = json.loads(body_parts_tracked_str)
        print(f"{section}. Processing videos with {body_parts_tracked}")
        if len(body_parts_tracked) > 5:
            body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]
    
        # We read all training data which match the body parts tracked
        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]

        _fps_lookup = (
            train_subset[['video_id', 'frames_per_second']]
            .drop_duplicates('video_id')
            .set_index('video_id')['frames_per_second']
            .to_dict()
        )

        single_mouse_list = []
        single_mouse_label_list = []
        single_mouse_meta_list = []

        mouse_pair_list = []
        mouse_pair_label_list = []
        mouse_pair_meta_list = []   

        for switch, data, meta, label in generate_mouse_data(train_subset, 'train'):
            if switch == 'single':
                single_mouse_list.append(data)
                single_mouse_meta_list.append(meta)
                single_mouse_label_list.append(label)
            else:
                mouse_pair_list.append(data)
                mouse_pair_meta_list.append(meta)
                mouse_pair_label_list.append(label)
            
            del data, meta, label

        gc.collect()
        
        # Construct a binary classifier
        binary_classifier = DataSampler(
            clone(CFG.model),
            neg_pos_ratio=10.0
        )
    
        # Predict single-mouse actions
        if len(single_mouse_list) > 0:
            # Concatenate all batches
            # The concatenation will generate label dataframes with missing values.

            # Xử lý data của các con chuột theo fps
            single_feats_parts = []
            for data_i, meta_i in zip(single_mouse_list, single_mouse_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup)
                X_i = transform_single(data_i, body_parts_tracked, fps_i).astype(np.float32)
                single_feats_parts.append(X_i)
                del fps_i, X_i
            gc.collect()

            X_tr = pd.concat(single_feats_parts, axis=0, ignore_index=True)
            single_mouse_label = pd.concat(single_mouse_label_list, axis=0, ignore_index=True)
            single_mouse_meta = pd.concat(single_mouse_meta_list, axis=0, ignore_index=True)
            
            del single_mouse_list, single_mouse_label_list, single_mouse_meta_list, single_feats_parts

            if validate_or_submit == 'validate':
                thresholds = cross_validate_classifier(X_tr, single_mouse_label, single_mouse_meta)
                if thresholds:
                    try:
                        if os.path.exists('thresholds.json'):
                            with open('thresholds.json', 'r') as f:
                                all_thresh = json.load(f)
                        else:
                            all_thresh = {}

                        section_key = str(body_parts_tracked_str)
                        if section_key not in all_thresh:
                            all_thresh[section_key] = {}

                        all_thresh[section_key].update(thresholds)

                        with open('thresholds.json', 'w') as f:
                            json.dump(all_thresh, f, indent=4)
                        print("  Saved thresholds to thresholds.json")
                    except Exception as e:
                        print(f"  Error saving thresholds: {e}")
            else:
                train_and_save_models(body_parts_tracked_str, 'single', X_tr, single_mouse_label, single_mouse_meta, section)
            del X_tr
        # Predict mouse-pair actions
        if len(mouse_pair_list) > 0:
            # Concatenate all batches
            # The concatenation will generate label dataframes with missing values.
            
            # Transform the coordinate representation into a distance representation for mouse_pair
            # Use a subset of body_parts_tracked to conserve memory

            pair_feats_parts = []
            for data_i, meta_i in zip(mouse_pair_list, mouse_pair_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup)
                X_i = transform_pair(data_i, body_parts_tracked, fps_i)
                pair_feats_parts.append(X_i)
                del X_i, fps_i
            gc.collect()

            X_tr = pd.concat(pair_feats_parts, axis=0, ignore_index=True)
            mouse_pair_label = pd.concat(mouse_pair_label_list, axis=0, ignore_index=True)
            mouse_pair_meta = pd.concat(mouse_pair_meta_list, axis=0, ignore_index=True)
            del mouse_pair_list, mouse_pair_label_list, mouse_pair_meta_list, pair_feats_parts

            if validate_or_submit == 'validate':
                thresholds = cross_validate_classifier(X_tr, mouse_pair_label, mouse_pair_meta)
                if thresholds:
                    try:
                        if os.path.exists('thresholds.json'):
                            with open('thresholds.json', 'r') as f:
                                all_thresh = json.load(f)
                        else:
                            all_thresh = {}

                        section_key = str(body_parts_tracked_str)
                        if section_key not in all_thresh:
                            all_thresh[section_key] = {}

                        all_thresh[section_key].update(thresholds)

                        with open('thresholds.json', 'w') as f:
                            json.dump(all_thresh, f, indent=4)
                        print("  Saved thresholds to thresholds.json")
                    except Exception as e:
                        print(f"  Error saving thresholds: {e}")
            else:
                train_and_save_models(body_parts_tracked_str, 'pair', X_tr, mouse_pair_label, mouse_pair_meta, section)
            del X_tr
        print("*"*50)
                
    except Exception as e:
        print(f'***Exception*** {e}')
    print()
if validate_or_submit != 'validate':
    if len(submission_list) > 0:
        submission = pd.concat(submission_list)
    else:
        submission = pd.DataFrame(
            dict(
                video_id=438887472,
                agent_id='mouse1',
                target_id='self',
                action='rear',
                start_frame=278,
                stop_frame=500
            ), index=[44])
    if validate_or_submit == 'submit':
        submission_robust = robustify(submission, test, 'test')
    submission_robust.index.name = 'row_id'
    submission_robust.to_csv('submission.csv')
    !head submission.csv